### reference

* https://learn.deeplearning.ai/courses/chatgpt-building-system/lesson/1/introduction

# Building Systems with the ChatGPT API

**ChatGPT API**를 사용하여 **시스템을 구축**하는 과정에 오신 것을 환영합니다.

**시스템을 구축**하는 과정에는 단일 프롬프트나 대규모 언어 모델에 대한 단일 호출보다 훨씬 더 많은 것이 필요합니다. 여기에서 **LLM을 사용**하여 **복잡한 애플리케이션을 구축**하기 위한 모범 **사례를 공유**하고자 합니다. 우리는 언어 모델에 대한 여러 호출을 연결하고 이전 호출의 출력에 따라 다른 명령을 사용하고 때로는 외부 소스에서 항목을 조회하는 **고객 지원 시스템을 구축**하는 실행 예제를 사용합니다.

예를 들어 "판매 중인 TV에 대해 알려주세요."와 같은 사용자 입력이 주어지면 다음 단계를 사용하여 이를 처리합니다.

* 먼저 **입력 내용을 평가**하여 증오심 표현과 같은 문제가 있는 콘텐츠가 포함되어 있지 않은지 확인할 수 있습니다. 
* 다음으로 시스템은 **입력을 처리**합니다. 이것이 **어떤 유형의 쿼리인지 식별**합니다. 불만 또는 제품 정보 요청 등입니까? 
* 제품 문의라고 판단되면 TV에 대한 관련 정보를 검색하고 **언어 모델을 사용하여 유용한 답변을 작성**합니다.
* 마지막으로 **출력을 확인**하여 부정확하거나 부적절한 답변과 같은 문제가 없는지 확인합니다. 

이 과정에서 확인할 수 있는 주제는 **응용 프로그램이 종종 최종 사용자에게 보이지 않는 여러 내부 단계를 필요로 한다는 것입니다**.

#  Language Models, the Chat Format and Tokens

## Large Language Model

##### Text generation process

| I love eating |  -------------------------------------------------- |
| ------------------|-------------------|
| [prompt] | bagles with cream cheese |
|        | my mothers's meatloaf |
|        | out with friends |

##### Supervised Learning (x --> y)

레스토랑 리뷰어 감정 분류

| Input x |  Output y |
| ------------------|-------------------|
| The pastromi sandwich was great | positive |
| Service was slow and the food was so~so | netative |
| The earl grey tea was fantastic. | positive |
| [Best pizza I've aver had!] | [positive] |


<img src="supervised.png" width="400">

##### Large Language Model (LLM) : How it works

지도 학습(x->y)을 사용하여 다음 단어를 반복적으로 예측하는 방식으로 언어 모델을 구축합니다.

My favorite food is a bagel with cream cheese and lox.

| Input x |  Output y |
| ------------------|-------------------|
| My favorite food is a | bagel |
| My favorite food is a bagel | with |
| My favorite food is a bagel with | cream |
| My favorite food is a bagel with cream | cheese |


##### Two types of large language models (LLMs)

**Base LLM**

텍스트 학습 데이터를 기반으로 다음 단어를 예측합니다.

*[Once up a time, there was a unicorn]*
<br>
that lived in a magical forest with all her unicorn friends.
<br>
<br>
*[What is the capital of France?]*
<br>
What is France's largest city?
<br>
What is France's population?
<br>
What is the currency of France?


**Instruction Tuned LLM**

지시를 따르려고 노력합니다

*[What is the capital of France?]*
<br>
The capital of France is Paries.

**Getting from a Base LLM to an Instruction Tuned LLM**
1. 많은 데이터에 대해 Base LLm을 훈련합니다.
2. 모델을 추가로 훈련합니다.
   - 입력 명령에 따라 출력을 하는 예들로 **미세 조정**합니다.
   - 도움이 되고(helpful), 정직하고(honest), 무해한(harmless)지 (HHH) 여부와 같은 기준에 따라 **다양한 LLM 결과물의 품질에 대한 인간 평가**를 얻습니다.
   - **LLM을 조정**하여 더 높은 등급의 출력을 생성할 확률을 높입니다(using RLHF: Reinforcement Learning from Human Feedback).

## Setup
#### Load the API key and relevant Python libaries.
이 과정에서는 OpenAI API 키를 로드하는 몇 가지 코드를 제공했습니다.

In [19]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

#### helper function


In [20]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 이것는 모델 출력의 무작위성 정도를 말합니다.[0~1]
    )
    return response.choices[0].message.content

## Prompt the model and get a completion

In [21]:
response = get_completion("What is the capital of France?")
print(response)

The capital of France is Paris.


## Tokens

In [22]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

pilpolol


In [23]:
response = get_completion("""Take the letters in l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


<img src="token.PNG" width=400>

영어 입력의 경우 1 token 은 약 4 글자, 즉 단어의 3/4 정도입니다.

token 한도
* 모델마다 입력 "컨텍스트" + 출력 완료의 토큰 수에 대한 제한이 다릅니다.
* gpt3.5-turbo ~ 4000 토큰

### System, User and Assistant Messages
```
messages =
[
 {"role": "system",
  "content": "You are an assistant ..."},
 {"role": "user",
  "content": "Tell me a joke "},
 {"role": "assistant",
  "content": "Why did the chicken ..."},
  ...
```
<img src="mesg.PNG" width=350>


## Helper function (chat format)


In [24]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

In [25]:
messages =  [  
{'role':'system', 
 'content':"You are an assistant who responds \
            in the style of Dr Seuss."},    
{'role':'user', 
 'content':"write me a very short poem about a happy carrot"},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh the happy carrot, so bright and so orange,
Dancing in the garden, free to explore and forage.
With a leafy green top and a cheerful grin,
This little veggie is sure to win. 
Beneath the sun, it grows and thrives,
A happy carrot, so full of life!


In [26]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a cheerful carrot named Charlie who always brought smiles to everyone in the garden.


In [27]:
# combined
messages =  [  
{'role':'system',
 'content':"You are an assistant who responds in the style of Dr Seuss. \
All your responses must be one sentence long."},    
{'role':'user',
 'content':"write me a story about a happy carrot"},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In a garden so bright, there lived a happy carrot, orange and upright.


In [28]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message.content
    
    token_dict = {
        'prompt_tokens':response.usage.prompt_tokens,
        'completion_tokens':response.usage.completion_tokens,
        'total_tokens':response.usage.total_tokens,
    }
    return content, token_dict

In [29]:
messages = [
{'role':'system', 
 'content':"You are an assistant who responds in the style of Dr Seuss."},    
{'role':'user',
 'content':"write me a very short poem about a happy carrot"},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [30]:
print(response)

Oh, the happy carrot, so bright and so bold,
In the garden, its story is joyfully told.
With a leafy green top and a vibrant orange hue,
It dances and sings, bringing smiles to you.

It grows in the earth, soaking up the sun,
Laughing and playing, oh what fun!
So here's to the carrot, so cheerful and merry,
Bringing happiness to all, let's give a big hurray!


In [31]:
print(token_dict)

{'prompt_tokens': 35, 'completion_tokens': 91, 'total_tokens': 126}
